# Generates crops ad pickle file using dataset_gen_pipe

This notebook crops images according to bounding box coordinates (one per sulcus)

# Imports

In [62]:
import sys
import os
import glob
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import the aims module
from soma import aims
# the brainplot package
import colorado as cld

print((sys.version))

3.6.9 (default, Jan 26 2021, 15:33:00) 
[GCC 8.4.0]


The following line permits to import deep_folding even if this notebook is executed from the notebooks subfolder (and no install has been launched):

 /notebooks/use_transform.ipynb  
 /deep_folding/__init__.py

In [63]:
sys.path.append((os.path.abspath('../')))
import deep_folding
print((os.path.dirname(deep_folding.__file__)))

/home/jc225751/Runs/18_test_visualization_marco/Program/deep_folding/deep_folding


In [64]:
_ALL_SUBJECTS = -1

# User-specific variables

In [65]:
sulcus = 'S.T.s.ter.asc.ant._left'

In [66]:
side = 'L'

We now assign path names and other user-specific variables.

The source directory is where the database lies. It contains the morphologist analysis subfolder ANALYSIS/3T_morphologist


In [67]:
src_dir = os.path.join(os.getcwd(), '../data/source/unsupervised')
src_dir = os.path.abspath(src_dir)
print(("src_dir = " + src_dir))

src_dir = /home/jc225751/Runs/18_test_visualization_marco/Program/deep_folding/data/source/unsupervised


The target directory tgt_dir is where the files will be saved

In [68]:
tgt_dir = os.path.join(os.getcwd(), '../data/target/data/linear')
tgt_dir = os.path.abspath(tgt_dir)
print(("tgt_dir = " + tgt_dir))

tgt_dir = /home/jc225751/Runs/18_test_visualization_marco/Program/deep_folding/data/target/data/linear


The reference directory is where the equivalent reference file has been saved

In [69]:
ref_dir = os.path.join(os.getcwd(), '../data/reference/data/linear')
ref_dir = os.path.abspath(ref_dir)
print(("ref_dir = " + ref_dir))

ref_dir = /home/jc225751/Runs/18_test_visualization_marco/Program/deep_folding/data/reference/data/linear


In [70]:
transform_dir = os.path.join(os.getcwd(), '../data/reference/transform')
transform_dir = os.path.abspath(transform_dir)
print(("transform_dir = " + transform_dir))

transform_dir = /home/jc225751/Runs/18_test_visualization_marco/Program/deep_folding/data/reference/transform


In [71]:
bbox_dir = os.path.join(os.getcwd(), '../data/reference/bbox')
bbox_dir = os.path.abspath(bbox_dir)
print(("bbox_dir = " + bbox_dir))

bbox_dir = /home/jc225751/Runs/18_test_visualization_marco/Program/deep_folding/data/reference/bbox


In [72]:
mask_dir = os.path.join(os.getcwd(), '../data/reference/mask')
mask_dir = os.path.abspath(mask_dir)
print(("mask_dir = " + mask_dir))

mask_dir = /home/jc225751/Runs/18_test_visualization_marco/Program/deep_folding/data/reference/mask


In [73]:
print((sys.argv))

['anatomist']


# Illustration of main program uses

We will first use the program with no effect by using number of subjects set to 0, or by calling the help function

### Using external calls

In [74]:
!python ../deep_folding/anatomist_tools/dataset_gen_pipe.py -n 0 -t tgt_local_dir

In [75]:
# Clean
!rm -rf tgt_local_dir

In [76]:
!python ../deep_folding/anatomist_tools/bounding_box.py --help

usage: bounding_box.py [-h] [-s SRC_DIR [SRC_DIR ...]] [-t TGT_DIR]
                       [-u SULCUS] [-i SIDE] [-m IMAGE_NORMALIZED_SPM]
                       [-p PATH_TO_GRAPH] [-n NB_SUBJECTS] [-v OUT_VOXEL_SIZE]

Computes bounding box around the named sulcus

optional arguments:
  -h, --help            show this help message and exit
  -s SRC_DIR [SRC_DIR ...], --src_dir SRC_DIR [SRC_DIR ...]
                        Source directory where the MRI data lies. If there are
                        several directories, add all directories one after the
                        other. Example: -s DIR_1 DIR_2. Default is :
                        /neurospin/lnao/PClean/database_learnclean/all/
  -t TGT_DIR, --tgt_dir TGT_DIR
                        Target directory where to store the output bbox json
                        files. Default is :
                        /neurospin/dico/data/deep_folding/test/bbox
  -u SULCUS, --sulcus SULCUS
                        Sulcus name around which 

### By using the main function call

In [77]:
from deep_folding.anatomist_tools import dataset_gen_pipe
print((dataset_gen_pipe.__file__))

/home/jc225751/Runs/18_test_visualization_marco/Program/deep_folding/deep_folding/anatomist_tools/dataset_gen_pipe.py


In [78]:
args = "-n 0 -t " + tgt_dir
argv = args.split(' ')

In [79]:
dataset_gen_pipe.main(argv)

In [80]:
args = "--help"
argv = args.split(' ')

In [81]:
dataset_gen_pipe.main(argv)

usage: dataset_gen_pipe.py [-h] [-s SRC_DIR] [-t TGT_DIR] [-a MASK_DIR]
                           [-b BBOX_DIR] [-m MORPHOLOGIST_DIR]
                           [-u SULCUS [SULCUS ...]] [-i SIDE] [-n NB_SUBJECTS]
                           [-e INTERP] [-p RESAMPLING] [-c CROPPING]
                           [-v OUT_VOXEL_SIZE [OUT_VOXEL_SIZE ...]]

Generates cropped and pickle files

optional arguments:
  -h, --help            show this help message and exit
  -s SRC_DIR, --src_dir SRC_DIR
                        Source directory where the MRI data lies. Default is :
                        /neurospin/hcp
  -t TGT_DIR, --tgt_dir TGT_DIR
                        Target directory where to store the cropped and pickle
                        files. Default is :
                        /neurospin/dico/data/deep_folding/test
  -a MASK_DIR, --mask_dir MASK_DIR
                        masking directory where mask has been stored. Default
                        is : /neurospin/dico/data/deep_

### By using the API function call

In [82]:
dataset_gen_pipe.dataset_gen_pipe(src_dir=src_dir,
                                  tgt_dir=tgt_dir,
                                  bbox_dir=bbox_dir,
                                  cropping='bbox',
                                  list_sulci=sulcus,
                                  side=side,
                                  number_subjects=0)

# Crops with linear interpolation

## Main program

In [83]:
interp = 'linear'

In [84]:
dataset_gen_pipe.dataset_gen_pipe(src_dir=src_dir,
                                  tgt_dir=tgt_dir,
                                  bbox_dir=bbox_dir,
                                  cropping='bbox',
                                  list_sulci=sulcus,
                                  side=side,
                                  interp=interp,
                                  number_subjects=1)

['100206']


## Result analysis

### Analysis of the inputs

In [85]:
# Gets source file as numpy array
skeleton_dir = os.path.join(src_dir, "ANALYSIS/3T_morphologist/100206/t1mri/default_acquisition/default_analysis/segmentation")
vol_source_file = glob.glob(skeleton_dir + '/' + side + '*.nii.gz')
vol_source = aims.read(vol_source_file[0])
arr_source = vol_source.arraydata()
print("shape of source skeleton = ", arr_source.shape)

shape of source skeleton =  (1, 260, 311, 260)


In [86]:
np.unique(arr_source)

array([ 0, 10, 11, 30, 40, 60, 80], dtype=int16)

In [87]:
pd.value_counts(np.resize(arr_source, arr_source.size))

11    18832738
0      2012736
60      163069
30        9634
80        5330
40          87
10           6
dtype: int64

### Analysis of the outputs

Prints the list of files of the target directory

In [88]:
print("Files in target directory:")
print(('\n'.join(os.listdir(tgt_dir))))

Files in target directory:
Ldataset.json
Lskeleton.pkl
Lcrops


In [89]:
print("Files in crops target directory:")
print(('\n'.join(os.listdir(tgt_dir + '/' + side + 'crops'))))

Files in crops target directory:
100206_normalized.nii.gz
100206_normalized.nii.gz.minf


In [90]:
tgt_json_file = glob.glob(tgt_dir + '/*.json')[0]
print("tgt_json_file = ", tgt_json_file, '\n')
with open(os.path.join(tgt_dir, tgt_json_file), 'r') as f:
    data_tgt = json.load(f)
    print((json.dumps(data_tgt, sort_keys=True, indent=4)))

tgt_json_file =  /home/jc225751/Runs/18_test_visualization_marco/Program/deep_folding/data/target/data/linear/Ldataset.json 

{
    "bbmax": [
        137,
        153,
        78
    ],
    "bbmin": [
        112,
        129,
        33
    ],
    "bbox_dir": "/home/jc225751/Runs/18_test_visualization_marco/Program/deep_folding/data/reference/bbox",
    "cropped_dir": "/home/jc225751/Runs/18_test_visualization_marco/Program/deep_folding/data/target/data/linear/Lcrops",
    "date": "2021-10-15 22:07:16",
    "git_sha": "c2104f396c5437b55ccc3799d9992de134d3f8c8",
    "interp": "linear",
    "is_git": true,
    "list_sulci": [
        "S.T.s.ter.asc.ant._left"
    ],
    "mask_dir": "/neurospin/dico/data/deep_folding/data/mask",
    "nb_subjects": 1,
    "out_voxel_size": [
        1,
        1,
        1
    ],
    "repo_working_dir": "/home/jc225751/Runs/18_test_visualization_marco/Program/deep_folding",
    "resampling_type": "AimsApplyTransform",
    "side": "L",
    "src_dir": "/ho

Obtained output (we read the cropped file from the target directory):

In [91]:
# Gets target crop as numpy array
cropped_target_dir = os.path.join(tgt_dir, side+'crops')
vol_target_file = glob.glob(cropped_target_dir + '/' + '*.nii.gz')
vol_target = aims.read(vol_target_file[0])
arr_target = vol_target.arraydata()
print("shape of target cropped image = ", arr_target.shape)

shape of target cropped image =  (1, 46, 25, 26)


In [92]:
np.unique(arr_target)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 68, 69], dtype=int16)

In [93]:
unique_target = np.unique(arr_target, return_counts=True)
print(unique_target)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 68, 69], dtype=int16), array([22026,   229,   171,   141,   129,   126,   111,    92,    98,
          85,    83,  3552,   118,   110,    95,   101,    84,    92,
         105,    86,    79,    79,    86,    86,    72,    95,    85,
          84,    93,   106,    86,   106,    92,   107,    74,    71,
          69,    57,    55,    58,    40,    43,    45,    49,    40,
          35,    41,    25,    36,    30,    28,    35,    22,    25,
          22,    22,    21,    21,    21,    12,     5,     4,     2,
           2]))


In [94]:
pd.value_counts(np.resize(arr_target, arr_target.size)).head()

0     22026
11     3552
1       229
2       171
3       141
dtype: int64

Expected output (we read the cropped file from the reference directory):

In [95]:
cropped_ref_dir = os.path.join(ref_dir, side+'crops')
vol_ref_file = glob.glob(cropped_ref_dir + '/' + '*.nii.gz')
vol_ref = aims.read(vol_ref_file[0])
arr_ref = vol_ref.arraydata()
print("shape of reference cropped image = ", arr_ref.shape)

shape of reference cropped image =  (1, 46, 25, 26)


In [96]:
pd.value_counts(np.resize(arr_ref, arr_ref.size)).head()

0     21572
11     3336
1       252
3       184
2       174
dtype: int64

In [97]:
np.array_equal(arr_target, arr_ref)

False

In [98]:
epsilon = 1
difference = (abs(arr_ref-arr_target) >= epsilon)
number_differences = np.count_nonzero(difference)
index_of_differences = np.where(difference)
print("Number of different pixels : ", number_differences)
print("Index of different pixels : ", index_of_differences)

Number of different pixels :  10576
Index of different pixels :  (array([0, 0, 0, ..., 0, 0, 0]), array([ 0,  0,  0, ..., 45, 45, 45]), array([ 0,  0,  0, ..., 24, 24, 24]), array([ 0,  1,  2, ..., 23, 24, 25]))


In [99]:
print(list(zip(arr_target[index_of_differences], arr_ref[index_of_differences])))

[(15, 21), (16, 0), (17, 0), (19, 0), (37, 0), (57, 6), (48, 30), (46, 0), (28, 0), (11, 0), (11, 0), (11, 0), (11, 0), (11, 0), (11, 0), (11, 4), (11, 30), (11, 0), (11, 0), (11, 0), (11, 0), (11, 0), (11, 0), (11, 37), (11, 13), (32, 19), (53, 29), (55, 5), (57, 0), (59, 0), (59, 8), (58, 27), (56, 0), (16, 0), (11, 0), (11, 0), (11, 0), (11, 0), (11, 0), (11, 0), (11, 0), (11, 1), (11, 0), (11, 0), (11, 0), (11, 0), (11, 0), (11, 15), (11, 27), (49, 0), (11, 16), (53, 29), (54, 26), (56, 7), (58, 10), (60, 25), (51, 0), (11, 0), (11, 0), (11, 0), (11, 0), (11, 0), (11, 0), (11, 0), (11, 0), (11, 0), (11, 0), (11, 0), (11, 0), (11, 0), (11, 0), (11, 35), (11, 16), (10, 0), (6, 0), (50, 0), (52, 14), (34, 37), (51, 35), (29, 7), (17, 0), (11, 0), (11, 0), (11, 0), (11, 0), (11, 0), (11, 0), (11, 0), (11, 0), (11, 0), (11, 0), (11, 0), (11, 0), (11, 0), (11, 3), (11, 44), (1, 0), (28, 0), (49, 0), (39, 43), (17, 6), (14, 0), (11, 0), (11, 0), (11, 0), (11, 0), (11, 0), (11, 0), (11, 0)

In [100]:
def are_arrays_almost_equal(arr1, arr2, epsilon, max_number_different_pixels):
    """Returns True if at most max_number_different_pixels pixels of arrays arr1 and arr2 
    differ by more than epsilon
    
    """
    difference = (abs(arr1-arr2) >= epsilon)
    number_different_pixels = np.count_nonzero(difference)
    return number_different_pixels <= max_number_different_pixels, number_different_pixels

In [101]:
equal_arrays, number_different_pixels = are_arrays_almost_equal(arr_ref, arr_target, 1, 2)

In [102]:
print(equal_arrays)

False


# Crops with mask and with nearest-neighbour interpolation

## Main program

In [103]:
tgt_dir_nearest = os.path.join(os.getcwd(), '../data/target/data/nearest')
tgt_dir_nearest = os.path.abspath(tgt_dir_nearest)
print(("tgt_dir = " + tgt_dir_nearest))

tgt_dir = /home/jc225751/Runs/18_test_visualization_marco/Program/deep_folding/data/target/data/nearest


In [104]:
interp = 'nearest'
dataset_gen_pipe.dataset_gen_pipe(src_dir=src_dir,
                                  tgt_dir=tgt_dir_nearest,
                                  mask_dir=mask_dir,
                                  cropping='mask',
                                  list_sulci=sulcus,
                                  side=side,
                                  interp=interp,
                                  number_subjects=1)

['100206']


## Result analysis

### Analysis of the inputs

In [105]:
# Gets source file as numpy array
skeleton_dir = os.path.join(src_dir, "ANALYSIS/3T_morphologist/100206/t1mri/default_acquisition/default_analysis/segmentation")
vol_source_file = glob.glob(skeleton_dir + '/' + side + '*.nii.gz')[0]
vol_source = aims.read(vol_source_file)
arr_source = vol_source.arraydata()
print("shape of source skeleton = ", arr_source.shape)

shape of source skeleton =  (1, 260, 311, 260)


In [106]:
np.unique(arr_source)

array([ 0, 10, 11, 30, 40, 60, 80], dtype=int16)

In [107]:
pd.value_counts(np.resize(arr_source, arr_source.size))

11    18832738
0      2012736
60      163069
30        9634
80        5330
40          87
10           6
dtype: int64

### Analysis of the outputs

Prints the list of files of the target directory

In [108]:
print("Files in crops target directory:")
print(tgt_dir_nearest)
print(('\n'.join(os.listdir(tgt_dir_nearest + '/' + side + 'crops'))))

Files in crops target directory:
/home/jc225751/Runs/18_test_visualization_marco/Program/deep_folding/data/target/data/nearest
100206_normalized.nii.gz
100206_normalized.nii.gz.minf


In [109]:
tgt_json_file = glob.glob(tgt_dir_nearest + '/*.json')[0]
print("tgt_json_file = ", tgt_json_file, '\n')
with open(os.path.join(tgt_dir_nearest, tgt_json_file), 'r') as f:
    data_tgt = json.load(f)
    print((json.dumps(data_tgt, sort_keys=True, indent=4)))

tgt_json_file =  /home/jc225751/Runs/18_test_visualization_marco/Program/deep_folding/data/target/data/nearest/Ldataset.json 

{
    "bbmax": [
        85,
        86,
        57,
        1
    ],
    "bbmin": [
        64,
        70,
        26,
        0
    ],
    "bbox_dir": "/neurospin/dico/data/deep_folding/data/bbox",
    "cropped_dir": "/home/jc225751/Runs/18_test_visualization_marco/Program/deep_folding/data/target/data/nearest/Lcrops",
    "date": "2021-10-15 22:07:20",
    "git_sha": "c2104f396c5437b55ccc3799d9992de134d3f8c8",
    "interp": "nearest",
    "is_git": true,
    "list_sulci": [
        "S.T.s.ter.asc.ant._left"
    ],
    "mask_dir": "/home/jc225751/Runs/18_test_visualization_marco/Program/deep_folding/data/reference/mask",
    "nb_subjects": 1,
    "out_voxel_size": [
        1,
        1,
        1
    ],
    "repo_working_dir": "/home/jc225751/Runs/18_test_visualization_marco/Program/deep_folding",
    "resampling_type": "AimsApplyTransform",
    "side": "L"

Obtained output (we read the cropped file from the target directory):

In [110]:
# Gets target crop as numpy array
cropped_target_dir = os.path.join(tgt_dir_nearest, side+'crops')
vol_target_file = glob.glob(cropped_target_dir + '/' + '*.nii.gz')
vol_target = aims.read(vol_target_file[0])
arr_target = vol_target.arraydata()
print("shape of target cropped image = ", arr_target.shape)

shape of target cropped image =  (1, 31, 16, 21)


In [111]:
np.unique(arr_target)

array([11], dtype=int16)

The scope here is to compare the different numbers present on the target array and on the source array:

In [112]:
np.around(pd.value_counts(np.resize(arr_target, arr_target.size))/arr_target.size*100, 1)

11    100.0
dtype: float64

In [113]:
np.around(pd.value_counts(np.resize(arr_source, arr_source.size))/arr_source.size*100, 1)

11    89.6
0      9.6
60     0.8
30     0.0
80     0.0
40     0.0
10     0.0
dtype: float64

### Visualization

In [114]:
import anatomist.notebook as ana
a = ana.Anatomist()
print(a.headless_info.__dict__)

{'xvfb': <subprocess.Popen object at 0x7f016d60b978>, 'original_display': ':1', 'display': 6, 'glx': 2, 'virtualgl': None, 'headless': True, 'mesa': False, 'qtapp': None}


In [115]:
def visualize_all_image(file_name):
    
    # load source skeleton data (the SliceableObject)
    object_anat = a.loadObject(file_name)

    # create an Axial window in anatomist
    w = a.createWindow("Axial", geometry=[1200, 350, 500, 500])
    object_anat.addInWindows(w)

    return object_anat, w

In [116]:
print("Files in crops nearest target directory:")
print(('\n'.join(os.listdir(tgt_dir_nearest + '/' + side + 'crops'))))

Files in crops nearest target directory:
100206_normalized.nii.gz
100206_normalized.nii.gz.minf


In [117]:
target_file_dir = tgt_dir_nearest + '/' + side + 'crops'
target_file_nearest = glob.glob(target_file_dir + "/*.nii.gz")[0]
print(target_file_nearest)

/home/jc225751/Runs/18_test_visualization_marco/Program/deep_folding/data/target/data/nearest/Lcrops/100206_normalized.nii.gz


In [118]:
visualize_all_image(target_file_nearest)

AnatomistInteractiveWidget(layout=Layout(height='auto', width='auto'), width=500)

(<anatomist.cpp.anatomist.SliceableObject object at 0x7f0133a775e8>,
 <anatomist.cpp.weak_shared_ptr_AWindow object at 0x7f0133a7ef78>)

In [119]:
target_file_dir = tgt_dir + '/' + side + 'crops'
target_file_linear = glob.glob(target_file_dir + "/*.nii.gz")[0]
print(target_file_linear)

/home/jc225751/Runs/18_test_visualization_marco/Program/deep_folding/data/target/data/linear/Lcrops/100206_normalized.nii.gz


In [120]:
visualize_all_image(target_file_linear)

AnatomistInteractiveWidget(layout=Layout(height='auto', width='auto'), width=500)

(<anatomist.cpp.anatomist.SliceableObject object at 0x7f016d54c558>,
 <anatomist.cpp.weak_shared_ptr_AWindow object at 0x7f0113fda0d8>)

In [121]:
visualize_all_image(vol_source_file)

AnatomistInteractiveWidget(layout=Layout(height='auto', width='auto'), width=500)

(<anatomist.cpp.anatomist.SliceableObject object at 0x7f0113fda948>,
 <anatomist.cpp.weak_shared_ptr_AWindow object at 0x7f0113ff3438>)